In [1]:
import pandas as pd
import openpyxl
import math
import statistics

# Repartiment d'escons per provincia segons el mètode

Diccionari amb el nombre de votants per provincia

In [2]:
votants = pd.read_excel("C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/Votants_per_provincia.xlsx")

provincies = list(votants.iloc[:,0])
escons_provincia = list(votants.iloc[:,8])

escons_per_provincia = {provincia:escons for provincia, escons in zip(provincies, escons_provincia)} 

A Espanya, el repartiment d'escons per provincia ve establert en base a la quota Hare com s'indica a (insertar referencia de la junta electoral)

El mètode D'Hont estableix que un partit pot obtenir un escon a la seva circumscirpció provincial només si supera el 3% dels vots totals. Així, els vots en blanc a l'únic que afecten és a incrementar el nombre de vots necessaris per obtenir un escon.

## Funcions

In [3]:
def quota(p, h):
    P = sum(p)
    q = [h*pi/P for pi in p]

    return q

def hamilton(p, h):
    # Calculam les quotes i els floors de les mateixes
    q = quota(p, h)
    b = [math.floor(qi) for qi in q]
    
    # Extreim la part decimal de les quotes
    dec = [qi-bi for (qi, bi) in zip(q, b)]

    # Guardam la posició de cada quota per saber a quina població correspon al realitzar l'ordenació
    dec_pos = [(d,i) for (d,i) in zip(dec, range(0, len(p)))]
    ordenada = sorted(dec_pos, key=lambda x: x[0], reverse = True)

    # Calculam els escons que queden a repartir
    rest = h -sum(b)

    # Afegim un escon a les h-sum(b) poblacions amb les quotes amb part decimal més gran
    for i in range(0, rest):
        b[ordenada[i][1]] += 1
        
    return b

In [10]:
def divisor_method(population, house, d):
    escons = [0]*len(population)

    # Mentre que els escons repartits no sigui igual a la mida de la cambra 
    while sum(escons) < house:
        # Calculam les poblacions entre d(el nombre d'escons que té cada estat fins al moment)
        populations_compare = [p/d(e) for p,e in 
                               zip(population,escons)]
        # Agafam l'estat que té el màxim a aqueix valor
        max_val = max(populations_compare)
        max_ind = populations_compare.index(max_val)
        # Sumam un escon a aquest estat
        escons[max_ind] += 1
    
    return escons

def d_adams(a):
    return a

def d_jefferson(a):
    return a + 1

def d_webster(a):
    return a + 1/2

def d_dean(a):
    return (a*(a+1))/(a + 1/2)

def d_hill(a):
    return math.sqrt(a*(a+1))

In [5]:
def divisor(p, h, d):
    # El primer divisor a provar és ds
    ds = sum(p)/h
    divisor = ds
    inf = 0
    sup = ds**10
    a = [0]*len(p)
    if d(0) == 0:
        if h <= len(p):
            for i in range(0, h):
                a[i] += 1
        else:
            a = [1]*len(p)
    
    i = 0
    while sum(a) != h and i <= 1000:
        i += 1
        # Calculam les quotes modificades
        q = [pi/divisor for pi in p]
        # Calculam el repartiment
        a = [math.floor(qi) if (d(math.floor(qi)-1) <= qi 
                                and qi <= d(math.floor(qi))) 
                                else math.ceil(qi) for qi in q]
        # Si la suma dels escons repartits és menor que la mida de la cambra, agafam un divisor menor
        if sum(a) < h:
            sup = divisor
            divisor = statistics.mean([inf,sup])
        # Si la suma dels escons a repartir és superior que la mida de la cambra, agafam un divisor superior
        if sum(a) > h:
            inf = divisor
            divisor = statistics.mean([inf,sup])

         
    return a

In [6]:
def Dhont(population, house):
    escons = [0]*len(population)
    while sum(escons) < house:
        populations_compare = [p/(e+1) for p,e in zip(population,escons)]

        max_val = max(populations_compare)
        max_ind = populations_compare.index(max_val)

        escons[max_ind] += 1
    
    return escons

In [41]:
def metode_quota(population, house):
    # Començam amb 0 escons
    escons = [0]*len(population)
    # Calculam les quotes de cada estat
    q = quota(population, house)
    
    # Mentre no estigui assignats tots els escons
    while sum(escons) < house:
        # calculam les poblacions entre el nombre d'escons que té cada estat fins al moment + 1
        populations_compare = [p/(e+1) for p,e 
                               in zip(population,escons)]

        # Agafam l'estat amb el màxim valor a la llista anterior
        max_val = max(populations_compare)
        max_ind = populations_compare.index(max_val)

        # Si assignar un escon no suposa violar la quota de l'estat, afegim l'escon a l'estat
        if escons[max_ind] <= math.floor(q[max_ind]):
            escons[max_ind] += 1

        # Si assignar un escon suposa violar la quota, no repartim cap escon més a aqueix estat
        else:
            population[max_ind] = 0

    
    return escons

In [39]:
p = [5117, 4400, 162, 161, 160]
h = 100

print(Dhont(p, h))
print(metode_quota(p, h))

[52, 45, 1, 1, 1]
[52, 44, 2, 1, 1]


Jefferson y D'Hont son exactament el mateix mètode.

In [42]:
p = [5117, 4400, 162, 161, 160]
h = 100

print(Dhont(p, h))
print(divisor_method(p, h, d_jefferson))
print(hamilton(p,h))
print(divisor(p,h,d_adams))
print(divisor_method(p,h,d_webster))
print(divisor(p,h,d_dean))
print(divisor(p,h,d_hill))
print(metode_quota(p, h))

[52, 45, 1, 1, 1]
[52, 45, 1, 1, 1]
[51, 44, 2, 2, 1]
[51, 43, 2, 2, 2]
[51, 43, 2, 2, 2]
[51, 43, 2, 2, 2]
[51, 43, 2, 2, 2]
[52, 44, 2, 1, 1]


### Càlcul d'escons pels diferents mètodes

In [44]:
# Llegim l'excel on tenim l'informació de les provincies
vots_provincia = 'C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/VOTS_PER_PROVINCIA_mes_3_percent.xlsx'

# Carregar l'arxiu excel
vots = pd.ExcelFile(vots_provincia)

# Nom de les fulles
noms_fulles = vots.sheet_names

vots_provincia_dhont = 'C:/Users/Pau Program/Documents/Mirror/UIB/MUSI/TFM/VOTS_PER_PROVINCIA_dhont.xlsx'


with pd.ExcelWriter(vots_provincia_dhont, 
                    engine='openpyxl') as writer:
    # Iterar sobre cada fulla
    for fulla in noms_fulles:
        # llegim la fulla
        df = vots.parse(fulla)
        print(fulla)

        # separam el nombre de vots a una llista
        p = list(df.iloc[:, 1])
        # veim el nombre d'escons que corresponen a la provincia
        h = int(escons_per_provincia[fulla])

        print(p, h)

        # calculam el nombre d'escons que correspon a cada partit 
        q = quota(p, h)
        escons_Dhont = Dhont(p, h)
        escons_hamilton = hamilton(p,h)
        print(fulla)
        escons_adams = divisor(p,h,d_adams)
        escons_webster = divisor_method(p,h,d_webster)
        escons_dean = divisor(p,h,d_dean)
        escons_hill = divisor(p,h,d_hill)
        escons_quota = metode_quota(p, h)

        df.loc[:, "QUOTA"] = q
        df.loc[:, "ESCONS D'HONT"] = escons_Dhont
        df.loc[:,"ESCONS HAMILTON"] = escons_hamilton
        df.loc[:,"ESCONS ADAMS"] = escons_adams
        df.loc[:,"ESCONS WEBSTER"] = escons_webster
        df.loc[:,"ESCONS DEAN"] = escons_dean
        df.loc[:,"ESCONS HILL"] = escons_hill
        df.loc[:,"ESCONS QUOTA"] = escons_quota

        # Escriure el dataframe a una nova fulla amb el nom actual de la fulla
        df.to_excel(writer, sheet_name=fulla, index=False)

A Coruña
[287997, 188184, 81345, 66996, 33606] 8
A Coruña
Alacant
[329167, 286424, 145562, 114973] 12
Alacant
Albacete
[88144, 75969, 36698, 15677] 4
Albacete
Almería
[130652, 92221, 67963, 21127] 6
Almería
Álava
[46476, 32844, 29985, 27936, 21407, 6500] 4
Álava
Asturias
[209581, 202280, 87534, 73438] 7
Asturias
Ávila
[42105, 26481, 14985, 7321, 4948] 3
Ávila
Badajoz
[152922, 147685, 53304, 26576] 5
Badajoz
Barcelona
[946055, 402527, 363857, 326388, 256231, 200138] 32
Barcelona
Bizkaia
[163508, 156639, 125418, 69771, 66189] 8
Bizkaia
Burgos
[81626, 69060, 25577, 17249] 4
Burgos
Cáceres
[91034, 89251, 31878, 16170] 4
Cáceres
Cádiz
[221020, 210596, 96391, 81544] 9
Cádiz
Cantabria
[145937, 115554, 48788, 29270] 5
Cantabria
Castelló
[107824, 99856, 48762, 43720] 5
Castelló
Ceuta
[12885, 11299, 7742] 1
Ceuta
Ciudad Real
[114443, 99800, 45942, 17256] 5
Ciudad Real
Córdoba
[169356, 143291, 62264, 61063] 6
Córdoba
Cuenca
[45159, 42358, 17660, 6282] 3
Cuenca
Gipuzkoa
[116414, 86711, 84338, 3943

In [9]:
print(noms_fulles)

['A Coruña', 'Alacant', 'Albacete', 'Almería', 'Álava', 'Asturias', 'Ávila', 'Badajoz', 'Barcelona', 'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria', 'Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Illes Balears', 'Jaén', 'La Rioja', 'Las Palmas', 'León', 'Lleida', 'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra', 'Ourense', 'Palencia', 'Pontevedra', 'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia', 'Valladolid', 'Zamora', 'Zaragoza']


In [ ]:
partits = []

for fulla in noms_fulles:
    df = vots.parse(fulla)
        
    # separam el nombre de vots a una llista
    p = list(df.iloc[:, 0])
    partits += p

print(set(partits))


{'CCa', 'PNV', 'SUMAR', 'NC-bc', 'JxCAT', 'CpM', 'EXISTE', 'B.N.G.', 'U.P.L.', 'EH Bildu', 'PP', 'U.P.N.', 'XAV', 'SY', 'PSOE', 'CUP', 'VOX', 'ERC'}
